In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
enddate = dt.datetime.now()
startdate = enddate - dt.timedelta(days = 365*5)

In [ ]:
stocks = ['GLO', 'SMC', 'SM', 'AC']

prices_df = pd.DataFrame()
for stock in stocks:
  hist = yf.download(stock, start = startdate, end = enddate)
  prices_df[stock] = hist['Close']

log_returns = np.log(prices_df/prices_df.shift(1))
log_returns = log_returns.dropna()

display(log_returns)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,GLO,SMC,SM,AC
Date,,,,
2019-10-30 00:00:00+00:00,0.005450,-0.022799,-0.049090,-0.001503
2019-10-31 00:00:00+00:00,-0.001088,0.004184,-0.013933,0.017642
2019-11-01 00:00:00+00:00,0.010823,0.028809,0.066609,0.002460
2019-11-04 00:00:00+00:00,0.002151,0.051395,0.139035,-0.004679
2019-11-05 00:00:00+00:00,-0.001075,0.020973,0.010331,-0.014421
...,...,...,...,...
2024-10-21 00:00:00+00:00,-0.003690,0.006415,-0.016708,-0.102272
2024-10-22 00:00:00+00:00,0.000000,-0.013998,0.004499,-0.066682
2024-10-23 00:00:00+00:00,-0.005561,-0.008778,-0.008303,-0.026728


In [ ]:
port = 100000
weights = np.array([1/len(stocks)] * len(stocks))

weighted_returns = (log_returns * weights).sum(axis = 1)

display(weighted_returns)

,0
Date,
2019-10-30 00:00:00+00:00,-0.016985
2019-10-31 00:00:00+00:00,0.001701
2019-11-01 00:00:00+00:00,0.027175
2019-11-04 00:00:00+00:00,0.046975
2019-11-05 00:00:00+00:00,0.003952
...,...
2024-10-21 00:00:00+00:00,-0.029064
2024-10-22 00:00:00+00:00,-0.019045
2024-10-23 00:00:00+00:00,-0.012342


In [ ]:
day = 5
n_day_return = weighted_returns.rolling(window = day).sum()
display(n_day_return)

,0
Date,
2019-10-30 00:00:00+00:00,NaN
2019-10-31 00:00:00+00:00,NaN
2019-11-01 00:00:00+00:00,NaN
2019-11-04 00:00:00+00:00,NaN
2019-11-05 00:00:00+00:00,0.062818
...,...
2024-10-21 00:00:00+00:00,-0.004054
2024-10-22 00:00:00+00:00,-0.012058
2024-10-23 00:00:00+00:00,-0.033324


In [ ]:
cov_matrix = log_returns.cov() *252
display(cov_matrix)

,GLO,SMC,SM,AC
GLO,0.076359,0.098779,0.091190,0.039176
SMC,0.098779,0.767339,0.387581,0.080123
SM,0.091190,0.387581,0.949508,0.090809
AC,0.039176,0.080123,0.090809,0.211270


In [ ]:

port_std_dev = np.sqrt(weights.T @ cov_matrix @ weights)
print(port_std_dev)

0.4730083632860479


In [ ]:
confidence_levels = [0.90, 0.95, 0.99]

VaRs = []
for cl in confidence_levels:
  VaR = port * norm.ppf(1 - cl) * port_std_dev * np.sqrt(day/252)
  VaRs.append(VaR)

print(VaRs)

[-8538.658350199556, -10959.249346050934, -15499.875490149763]
